# stocks 정보 최신화

In [1]:
# pykrx to mysql
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from openpyxl import load_workbook
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

start = '20190101'
end = '20211018'
# end = str(datetime.now().date()).replace('-', '')
period = pd.date_range(start=start, end=end).strftime("%Y%m%d").tolist()
markets = ['KOSDAQ']

for date in period:
  for market in markets:
    try:
      db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
      conn = db_connection.connect()
      df = stock.get_market_ohlcv_by_ticker(date, market=market)
      time.sleep(0.5)
      if (df['시가'][0] == 0):
        continue
      df['date'] = date
      df = df.reset_index().set_index('date').drop(['거래대금', '등락률'], axis=1).rename(columns={'티커':'ticker', '시가':'open', '고가':'high', '저가':'low', '종가':'close', '거래량':'volume'})
      df.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
      conn.close()
    except Exception as ex:
      print(ex, date)
      pass


# 볼린저밴드 구하기

In [3]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      # print(ticker[0], date[0].strftime("%Y%m%d"))
      try:
        getLast20InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit 20"
        cursor.execute(getLast20InfoSql)
        last20InfoDf = pd.DataFrame(cursor.fetchall())
        if len(last20InfoDf.index) < 20:
          continue
        if last20InfoDf.iloc[0][6] == 0:
          continue
        
        last20InfoDfClose = last20InfoDf.loc[:, 5]
        avg = last20InfoDfClose.mean()
        std = last20InfoDfClose.std()
        tmpInfo = last20InfoDf.loc[0, [0, 1, 5]]
        hi = round(avg + std * 2)
        me = round(avg)
        lo = round(avg - std * 2)
        bw = (hi - lo) / me
        pos = (tmpInfo[5] - lo) / (hi - lo)

        insertBollSql = "REPLACE INTO boll(date, ticker, close, low, medium, high, bandWidth, position) VALUES(%s, %s, %s, %s, %s, %s, %s, %s)" 
        cursor.execute(insertBollSql, (tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw, pos))
        db.commit()
      except Exception as ex1:
        print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"), tmpInfo[0], tmpInfo[1], tmpInfo[5], lo, me, hi, bw, pos)
        # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

C:\Users\Jack\AppData\Local\Temp/ipykernel_8144/3470133148.py:43: RuntimeWarning: invalid value encountered in longlong_scalars
  pos = (tmpInfo[5] - lo) / (hi - lo)


ex1 (1366, "Incorrect double value: 'nan' for column `INDEX_DUCK`.`boll`.`position` at row 1") 083380 20190318 2019-03-18 083380 5010 5010 5010 5010 0.0 nan


# Z-SCORE
- (현재값 - 평균) / 표준편차

In [127]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

# n_days = [10, 20, 30, 90, 180]
n_days = [180]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLastNBollSql = "SELECT * FROM boll WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day)
          cursor.execute(getLastNBollSql)
          lastNBollDf = pd.DataFrame(cursor.fetchall())
          if len(lastNBollDf.index) < day:
            continue

          lastNBollBandWidthDf = lastNBollDf.loc[:, 6]
          avg = lastNBollBandWidthDf.mean()
          std = lastNBollBandWidthDf.std()
          tmpInfo = lastNBollDf.loc[0, [0, 1, 6]]
          z_score = (tmpInfo[6] - avg) / std
          
          insertZScoreSql = "REPLACE INTO z_score(date, ticker, period, bandWidth, z_score) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertZScoreSql, (tmpInfo[0], tmpInfo[1], day, tmpInfo[6], z_score))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          # print(tmpInfo[0], tmpInfo[1], tmpInfo[2], hi, me, lo, bw)
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# Enhancing Z-score

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

# n_days = [10, 20, 30, 90, 180]
n_days = [180]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:
    for day in n_days:
      try:
        # Get ticker's bollinger
        cursor.execute("SELECT * FROM boll WHERE ticker = '" + ticker[0] + " ORDER BY date DESC")
        bollDf = pd.DataFrame(cursor.fetchall())
        bollDf = bollDf.rename(columns={0: 'date', 1:'ticker', 2:'close', 3:'low', 4:'medium', 5:'high', 6:'bandWidth', 7:'position'})
        bollDf['period'] = day
        bollDf['z_score'] = 0
        # Set z_score
        for idx in range(0, len(bollDf)):
          if idx + day > len(bollDf) - 1:
            continue
          copyDf = bollDf[idx:idx+day].loc[:, 'bandWidth'].copy()
          avg = copyDf.mean()
          std = copyDf.std()
          bollDf.at[idx, 'z_score'] = (bollDf.iloc[idx]['bandWidth'] - avg) / std
        bollDf = bollDf.reset_index().set_index('date').drop(['index', 'close', 'low', 'medium', 'high', 'posision'], axis=1)
        # db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
        # conn = db_connection.connect()
        # bollDf.to_sql(name='z_score', con=db_connection, if_exists='append', index=True, index_label="date")        
        # conn.close()
      except Exception as ex1:
        print('ex1', ex1, ticker[0])
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# MFI
- TP(Typical price) = (당일 고가 + 당일 저가 + 당일 종가) / 3
- RMF = TP * 당일 거래량
  - Positive RMF: 당일 TP > 전일 TP
  - Negative RMF: 당일 TP < 전일 TP
- MFI = n일간 positive RMF 합계 / (n일간 positive RMF 합계 + n일간 negative RMF 합계) * 100

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  getPeriodSql = "SELECT DISTINCT date FROM stocks_price ORDER BY date DESC"
  cursor.execute(getPeriodSql)
  period = list(cursor.fetchall())

  for ticker in tickers:
    for date in period:
      for day in n_days:
        # print(ticker[0], date[0].strftime("%Y%m%d"))
        try:
          getLast10InfoSql = "SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' AND date <= '" + date[0].strftime("%Y%m%d") + "' ORDER BY date DESC limit " + str(day + 1)
          cursor.execute(getLast10InfoSql)
          last10InfoDf = pd.DataFrame(cursor.fetchall())
          if len(last10InfoDf.index) < day + 1:
            continue
          tpList = []
          for idx in last10InfoDf.index:
            info = last10InfoDf.loc[idx]
            tpList.append(round((info[3] + info[4] + info[5]) / 3))
          last10InfoDf['TP'] = tpList

          positiveRMF = 0
          negativeRMF = 0
          for idx in range(0, len(last10InfoDf) - 1):
            today = last10InfoDf.iloc[idx]
            yesterday = last10InfoDf.iloc[idx + 1]
            if today['TP'] > yesterday['TP']:
              # positive
              positiveRMF += today['TP'] * today[6]
            elif today['TP'] < yesterday['TP']:
              # negative
              negativeRMF += today['TP'] * today[6]
          
          MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
          todayInfo = last10InfoDf.iloc[0, [0, 1, 7]]

          insertBollSql = "REPLACE INTO mfi(date, ticker, period, tp, mfi) VALUES(%s, %s, %s, %s, %s)"
          cursor.execute(insertBollSql, (todayInfo[0], todayInfo[1], day, todayInfo['TP'], MFI))
          db.commit()
        except Exception as ex1:
          print('ex1', ex1, ticker[0], date[0].strftime("%Y%m%d"))
          pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()

# Enhancing MFI

In [69]:
import pandas as pd
import pymysql
import datetime
from sqlalchemy import create_engine

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
PW = 'root'
# PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

n_days = [10]

try:
  getTickersSql = "SELECT DISTINCT ticker FROM stocks_price"
  cursor.execute(getTickersSql)
  tickers = list(cursor.fetchall())

  for ticker in tickers:
    for day in n_days:
      try:
        # Get ticker's price
        cursor.execute("SELECT * FROM stocks_price WHERE ticker = '" + ticker[0] + "' ORDER BY date DESC")
        priceDf = pd.DataFrame(cursor.fetchall())
        priceDf = priceDf.rename(columns={0: 'date', 1:'ticker', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
        priceDf['period'] = day
        priceDf['tp'] = 0
        priceDf['mfi'] = 0
        # Set typical price
        for idx in range(0, len(priceDf)):
          row = priceDf.iloc[idx]
          priceDf.at[idx, 'tp'] = round((row['high'] + row['low'] + row['close']) / 3)
        # Set MFI
        for idx in range(0, len(priceDf)):
          if idx + day > len(priceDf) - 1:
            continue
          positiveRMF = 0
          negativeRMF = 0          
          for i in range(idx, idx + day):
            today = priceDf.iloc[i]
            yesterday = priceDf.iloc[i + 1]
            if today['tp'] > yesterday['tp']:
              positiveRMF += today['tp'] * today['volume']
            elif today['tp'] < yesterday['tp']:
              negativeRMF += today['tp'] * today['volume']
          MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
          priceDf.at[idx, 'mfi'] = MFI
        priceDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
        db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8')
        conn = db_connection.connect()
        priceDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
        conn.close()
      except Exception as ex1:
        print('ex1', ex1, ticker[0])
        pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()
# priceDf.head()

ex1 (pymysql.err.IntegrityError) (1062, "Duplicate entry '000020-2021-10-18-10' for key 'PRIMARY'")
[SQL: INSERT INTO mfi (date, ticker, period, tp, mfi) VALUES (%(date)s, %(ticker)s, %(period)s, %(tp)s, %(mfi)s)]
[parameters: ({'date': datetime.date(2021, 10, 18), 'ticker': '000020', 'period': 10, 'tp': 17983, 'mfi': 82}, {'date': datetime.date(2021, 10, 15), 'ticker': '000020', 'period': 10, 'tp': 18533, 'mfi': 87}, {'date': datetime.date(2021, 10, 14), 'ticker': '000020', 'period': 10, 'tp': 18917, 'mfi': 90}, {'date': datetime.date(2021, 10, 13), 'ticker': '000020', 'period': 10, 'tp': 18467, 'mfi': 84}, {'date': datetime.date(2021, 10, 12), 'ticker': '000020', 'period': 10, 'tp': 19067, 'mfi': 86}, {'date': datetime.date(2021, 10, 8), 'ticker': '000020', 'period': 10, 'tp': 19700, 'mfi': 90}, {'date': datetime.date(2021, 10, 7), 'ticker': '000020', 'period': 10, 'tp': 19400, 'mfi': 90}, {'date': datetime.date(2021, 10, 6), 'ticker': '000020', 'period': 10, 'tp': 18650, 'mfi': 88} 

C:\Users\Jack\AppData\Local\Temp/ipykernel_8144/1024054416.py:50: RuntimeWarning: invalid value encountered in longlong_scalars
  MFI = positiveRMF / (positiveRMF + negativeRMF) * 100


ex1 Cannot assign nan to integer series 002300
ex1 division by zero 002380
ex1 division by zero 002420
ex1 division by zero 002630
ex1 division by zero 002840
ex1 division by zero 003280
ex1 division by zero 003410
ex1 division by zero 003415
ex1 division by zero 003550
ex1 division by zero 003555
ex1 division by zero 003620
ex1 division by zero 004150
ex1 division by zero 004200
ex1 division by zero 004540
ex1 division by zero 004545
ex1 division by zero 004920
ex1 division by zero 004987
ex1 division by zero 004989
ex1 division by zero 005190
ex1 Cannot assign nan to integer series 005420
ex1 division by zero 005450
ex1 division by zero 005850
ex1 division by zero 006580
ex1 division by zero 006980
ex1 division by zero 007120
ex1 division by zero 007530
ex1 division by zero 007630
ex1 division by zero 007700
ex1 division by zero 008060
ex1 division by zero 00806K
ex1 division by zero 008800
ex1 division by zero 009410
ex1 division by zero 009415
ex1 division by zero 009620
ex1 divisi

# bollinger squeeze back test

In [ ]:
import pandas as pd
import pymysql
import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'INDEX_DUCK'
ID = 'root'
# PW = 'root'
PW = 'test'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

# 'market': ['KOSPI', 'KOSDAQ'],
# 'position_bid': [0.7, 0.8, 0.9],
# 'position_ask': [0.1, 0.2, 0.3],
# 'mfi_bid': [70, 80, 90],
# 'mfi_ask': [10, 20, 30],
# 'z_score': [-1.0, -1.5, -2.0],
# 'z_score_period': [10, 20, 30, 90, 180]

## Back test 파라미터
# 기간
start = "20200101"
end = "20211001"
# 볼린저밴드 %b 매수/매도
position_bid = 0.8
position_ask = 0.2
# 볼린저밴드 z score
z_score = -1.3
# 볼린저밴드 z score period
z_score_period = 180
# MFI
mfi_bid = 80
mfi_ask = 20
# Market
market = 'KOSPI'
# 투자금
amount = 1000000

try:
  # Get ticker list
  # cursor.execute("SELECT DISTINCT ticker FROM stocks_price")
  if market != 'ALL':
    cursor.execute("SELECT DISTINCT ticker FROM stocks_info WHERE market='" + market + "'")
  else:
    cursor.execute("SELECT DISTINCT ticker FROM stocks_info")
  tickers = list(cursor.fetchall())
  # Get period list
  cursor.execute("SELECT DISTINCT date FROM stocks_price WHERE date >= '"+ start +"' AND date <= '"+ end +"' ORDER BY date")
  period = list(cursor.fetchall())
  totalHistory = []
  totalPNL = 0
  history = {}
  winCnt, loseCnt, winRate = 0, 0, 0
  for ticker in tickers:
    try:
      # Get bollinger band dataframe
      cursor.execute("SELECT * FROM boll WHERE ticker = '" + ticker[0] + "'  ORDER BY date")
      bollDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'close', 3:'low', 4:'medium', 5:'high', 6:'bandWidth', 7:'position'})
      bollDf.set_index('date', drop=True, inplace=True)
      # Get MFI dataframe
      cursor.execute("SELECT * FROM mfi WHERE ticker = '" + ticker[0] + "' ORDER BY date")
      mfiDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'tp', 4:'mfi'})      
      mfiDf.set_index('date', drop=True, inplace=True)
      mfiDf = mfiDf.drop(['ticker', 'period'], axis=1)
      # Get z-score dataframe
      cursor.execute("SELECT * FROM z_score WHERE ticker = '" + ticker[0] + "' AND period = '" + str(z_score_period) + "' ORDER BY date")
      zscoreDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'ticker', 2:'period', 3:'bandWidth', 4:'z_score'})      
      zscoreDf = zscoreDf.drop(['ticker', 'period', 'bandWidth'], axis=1)
      zscoreDf.set_index('date', drop=True, inplace=True)
      sumDf = pd.concat([bollDf, mfiDf, zscoreDf], axis=1)
      isBid = False
      bidPrice = 0
      bidDate = '000000'      
      for idx in sumDf.index:
        row = sumDf.loc[idx].copy()
        if pd.isna(row['z_score']):
          continue
        # Bid Step
        if not isBid:
          if row['position'] >= position_bid and row['mfi'] >= mfi_bid and row['z_score'] < z_score:
            row['status'] = 'bid'
            totalHistory.append(row)
            isBid = True
            bidPrice = row['close']
            bidDate = row.name.strftime("%Y%m%d").tolist()
        # Ask Step
        else:
          if row['position'] <= position_ask or row['mfi'] <= mfi_ask:
            row['status'] = 'ask'
            totalHistory.append(row)
            isBid = False
            pnl = (1 - (bidPrice / row['close'])) * 100
            if pnl > 0:
              winCnt += 1
            else:
              loseCnt += 1
            totalPNL += pnl
            if ticker[0] in history:
              history[ticker[0]].append( bidDate + " bid: " + str(bidPrice) + ", " + row.name.strftime("%Y%m%d").tolist() + " ask: " + str(row['close']) + ", pnl: " + str(pnl))
            else:
              history[ticker[0]] = [bidDate + " bid: " + str(bidPrice) + ", " + row.name.strftime("%Y%m%d").tolist() + " ask: " + str(row['close']) + ", pnl: " + str(pnl)]
      if isBid:
        idxLst = list(sumDf.index).sort()
        row = sumDf.loc[idxLst[-1]].copy()
        totalPNL += (1 - (bidPrice / row['close'])) * 100      
        if ticker[0] in history:
          history[ticker[0]].append(bidDate + " bid: " + str(bidPrice) + ", having: " + str(row['close']) + ", pnl: " + str(pnl))
        else:
          history[ticker[0]] = [bidDate + " bid: " + str(bidPrice) + ", having: " + str(row['close']) + ", pnl: " + str(pnl)]
      # insertBollSql = "REPLACE INTO mfi(date, ticker, period, tp, mfi) VALUES(%s, %s, %s, %s, %s)"
      # cursor.execute(insertBollSql, (todayInfo[0], todayInfo[1], day, todayInfo['TP'], MFI))
      # db.commit()
    except Exception as ex1:
      print('ex1', ex1, ticker[0])
      pass
except Exception as ex2:
  print('ex2', ex2)
  pass

db.close()
winRate = (winCnt / (winCnt + loseCnt)) * 100

# 파라미터 최적화

In [ ]:
import itertools

param_grid = {
              'market': ['KOSPI', 'KOSDAQ'],
              'position_bid': [0.7, 0.8, 0.9],
              'position_ask': [0.1, 0.2, 0.3],
              'mfi_bid': [70, 80, 90],
              'mfi_ask': [10, 20, 30],
              'z_score': [-1.0, -1.5, -2.0],
              'z_score_period': [10, 20, 30, 90, 180]
              }

# generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
